In [1]:
library(tidyverse)
library(RPostgreSQL)

Loading tidyverse: ggplot2
Loading tidyverse: tibble
Loading tidyverse: tidyr
Loading tidyverse: readr
Loading tidyverse: purrr
Loading tidyverse: dplyr
Conflicts with tidy packages ---------------------------------------------------
filter(): dplyr, stats
lag():    dplyr, stats
Loading required package: DBI


In [2]:
sql_dir <- file.path("..", "SQL")
list.files(sql_dir)

[1] "cohort.sql"     "data.sql"       "population.sql"

In [3]:
drv <- dbDriver("PostgreSQL")
con <- dbConnect(drv, dbname = "mimic", user = "lcp_db_user")
dbSendQuery(con, "set search_path=echo,public,mimiciii;")

<PostgreSQLResult>

#### Angus cohort

In [4]:
dbGetQuery(con, "select count(*) from population where angus = 1;")

count
17420


#### Adult in their 1st ICU stay

In [5]:
sql <- "
select count(*)
  from population
 where angus = 1
   and age >= 18
   and icu_order = 1
"
dbGetQuery(con, sql)

count
10566


#### MICU/SICU only

In [6]:
sql <- "
select count(*)
  from population
 where angus = 1
   and age >= 18
   and icu_order = 1
   and first_careunit in ('MICU', 'SICU')
   and (echo_time is null or (echo_time <= outtime
   and echo_time >= intime))
"
dbGetQuery(con, sql)

count
5517


In [7]:
sql <- "
select count(*)
  from population
 where angus = 1
   and age >= 18
   and icu_order = 1
   and first_careunit in ('MICU', 'SICU')
   and echo_time <= outtime
   and echo_time >= intime
"
dbGetQuery(con, sql)

count
2418


In [8]:
dbDisconnect(con)
dbUnloadDriver(drv)

[1] TRUE

[1] TRUE